# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FELIPE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FELIPE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\FELIPE\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import libraries
import os
import re
import sys
import pickle
import datetime
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from scipy import stats

from scipy.stats import gmean

# import relevant functions/modules from the sklearn
from nltk.corpus import stopwords 
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier

In [3]:
# load data from database
database_filepath = "../data/disaster_response_db.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
df = pd.read_sql_table(table_name,engine)

In [4]:
print('the dataset has', df.shape[0], 'rows and', df.shape[1], 'columns.')

the dataset has 26216 rows and 40 columns.


In [5]:
# A quick look of the data
df.describe()

id       related       request         offer   aid_related  \
count  26216.00000  26216.000000  26216.000000  26216.000000  26216.000000   
mean   15224.82133      0.773650      0.170659      0.004501      0.414251   
std     8826.88914      0.435276      0.376218      0.066940      0.492602   
min        2.00000      0.000000      0.000000      0.000000      0.000000   
25%     7446.75000      1.000000      0.000000      0.000000      0.000000   
50%    15662.50000      1.000000      0.000000      0.000000      0.000000   
75%    22924.25000      1.000000      0.000000      0.000000      1.000000   
max    30265.00000      2.000000      1.000000      1.000000      1.000000   

       medical_help  medical_products  search_and_rescue      security  \
count  26216.000000      26216.000000       26216.000000  26216.000000   
mean       0.079493          0.050084           0.027617      0.017966   
std        0.270513          0.218122           0.163875      0.132831   
min        0.000000          0.000000           0.000000      0.000000   
25%        0.000000          0.000000           0.000000      0.000000   
50%        0.000000          0.000000           0.000000      0.000000   
75%        0.000000          0.000000           0.000000      0.000000   
max        1.000000          1.000000           1.000000      1.000000   

           military  ...   aid_centers  other_infrastructure  weather_related  \
count  26216.000000  ...  26216.000000          26216.000000     26216.000000   
mean       0.032804  ...      0.011787              0.043904         0.278341   
std        0.178128  ...      0.107927              0.204887         0.448191   
min        0.000000  ...      0.000000              0.000000         0.000000   
25%        0.000000  ...      0.000000              0.000000         0.000000   
50%        0.000000  ...      0.000000              0.000000         0.000000   
75%        0.000000  ...      0.000000              0.000000         1.000000   
max        1.000000  ...      1.000000              1.000000         1.000000   

             floods         storm          fire    earthquake          cold  \
count  26216.000000  26216.000000  26216.000000  26216.000000  26216.000000   
mean       0.082202      0.093187      0.010757      0.093645      0.020217   
std        0.274677      0.290700      0.103158      0.291340      0.140743   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      0.000000      0.000000   
max        1.000000      1.000000      1.000000      1.000000      1.000000   

       other_weather  direct_report  
count   26216.000000   26216.000000  
mean        0.052487       0.193584  
std         0.223011       0.395114  
min         0.000000       0.000000  
25%         0.000000       0.000000  
50%         0.000000       0.000000  
75%         0.000000       0.000000  
max         1.000000       1.000000  

[8 rows x 37 columns]

In [6]:
print("Summary of the basic information about this dataFrame and its data:")
print(df.info())

Summary of the basic information about this dataFrame and its data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26216 non-null  int64 
 1   message                 26216 non-null  object
 2   original                10170 non-null  object
 3   genre                   26216 non-null  object
 4   related                 26216 non-null  int64 
 5   request                 26216 non-null  int64 
 6   offer                   26216 non-null  int64 
 7   aid_related             26216 non-null  int64 
 8   medical_help            26216 non-null  int64 
 9   medical_products        26216 non-null  int64 
 10  search_and_rescue       26216 non-null  int64 
 11  security                26216 non-null  int64 
 12  military                26216 non-null  int64 
 13  child_alone             26216 non-null

In [7]:
print("Columns names of the dataframe:")
df.columns

Columns names of the dataframe:


Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [8]:
# Dropping 'child_alone' column
df = df.drop(['child_alone'],axis=1)

In [9]:
df["related"].replace({2: 0}, inplace=True)

In [10]:
# Selecting variables
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [11]:
print("X shape:")
X.shape

X shape:


(26216,)

In [12]:
print("y shape:")
y.shape

y shape:


(26216, 35)

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text, urlplaceholder="urlplaceholder"):
    """
    Function: split text into words and return the root form of the words
    Inputs:
      text: text messages
    Outputs:
      root_words: a list of the root form of the message words
    """
    # 1. Replace all urls with a url place holder string
    url_rgx = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # 2. Extract all the urls from text 
    urls = re.findall(url_rgx, text)
    
    # 3. Replace url with a url placeholder string
    for url in urls:
        text = text.replace(url, urlplaceholder)

    # 4. Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # 5. Lemmatization
    lemmat = nltk.WordNetLemmatizer()

    # 6. List of clean tokens
    root_words = [lemmat.lemmatize(w).lower().strip() for w in tokens]
    return root_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
# Pipeline 1: Adaboost Classifier 
pipeline_ada = Pipeline([
        ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
        ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
        ('tfidf_transformer', TfidfTransformer())]))
        ])),
        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
pipeline_ada_fit = pipeline_ada.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred_train = pipeline_ada_fit.predict(X_train)
y_pred_test  = pipeline_ada_fit.predict(X_test)

In [18]:
# training data
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.83      0.94      0.88     14915
               request       0.80      0.56      0.66      3367
                 offer       0.35      0.07      0.12        80
           aid_related       0.77      0.62      0.68      8174
          medical_help       0.64      0.28      0.39      1559
      medical_products       0.71      0.35      0.47       993
     search_and_rescue       0.69      0.23      0.34       540
              security       0.40      0.06      0.11       337
              military       0.66      0.37      0.47       660
                 water       0.79      0.65      0.71      1229
                  food       0.81      0.69      0.75      2184
               shelter       0.80      0.58      0.67      1739
              clothing       0.78      0.46      0.58       305
                 money       0.65      0.34      0.45       447
        missing_people       0.68    

C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
# test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      4991
               request       0.77      0.57      0.65      1107
                 offer       0.08      0.03      0.04        38
           aid_related       0.76      0.61      0.68      2686
          medical_help       0.63      0.30      0.41       525
      medical_products       0.62      0.32      0.42       320
     search_and_rescue       0.58      0.20      0.29       184
              security       0.18      0.02      0.04       134
              military       0.63      0.37      0.47       200
                 water       0.76      0.62      0.68       443
                  food       0.78      0.70      0.74       739
               shelter       0.74      0.56      0.64       575
              clothing       0.72      0.46      0.56       100
                 money       0.54      0.22      0.32       157
        missing_people       0.57      

C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
print("Parameters of the AdaBoostClassifier Pipeline")
pipeline_ada.get_params().keys()

Parameters of the AdaBoostClassifier Pipeline


dict_keys(['memory', 'steps', 'verbose', 'features', 'classifier', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__count_vectorizer', 'features__text_pipeline__tfidf_transformer', 'features__text_pipeline__count_vectorizer__analyzer', 'features__text_pipeline__count_vectorizer__binary', 'features__text_pipeline__count_vectorizer__decode_error', 'features__text_pipeline__count_vectorizer__dtype', 'features__text_pipeline__count_vectorizer__encoding', 'features__text_pipeline__count_vectorizer__input', 'features__text_pipeline__count_vectorizer__lowercase', 'features__text_pipeline__count_vectorizer__max_df', 'features__text_pipeline__count_vectorizer__max_features', 'features__text_pipeline__count_vectorizer__min_df', 'features__text_pipeline__count_vectorizer__ngram_range', '

In [21]:
param_ada = {'classifier__estimator__learning_rate': [0.01, 0.03, 0.05],
             'classifier__estimator__n_estimators': [10, 30, 50]}

cv_ada = GridSearchCV(pipeline_ada, param_grid=param_ada, scoring='f1_micro', n_jobs=-1)

In [22]:
cv_ada.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('count_vectorizer',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x0000020DFC9B0AF0>)),
                                                                                        ('tfidf_transformer',
                                                                                         TfidfTransformer())]))])),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__learning_rate': [0.01, 0.03,
                                                                  0.05],
                         'classifier_

In [23]:
y_pred_test  = cv_ada.predict(X_test)
y_pred_train = cv_ada.predict(X_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [24]:
# training data
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.76      1.00      0.86     14915
               request       0.60      0.38      0.47      3367
                 offer       1.00      0.03      0.05        80
           aid_related       0.80      0.19      0.30      8174
          medical_help       0.64      0.11      0.19      1559
      medical_products       0.75      0.13      0.22       993
     search_and_rescue       0.61      0.19      0.29       540
              security       0.00      0.00      0.00       337
              military       0.29      0.00      0.01       660
                 water       0.57      0.86      0.69      1229
                  food       0.78      0.68      0.72      2184
               shelter       0.84      0.29      0.43      1739
              clothing       0.76      0.32      0.45       305
                 money       0.59      0.19      0.29       447
        missing_people       0.68    

C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4991
               request       0.63      0.40      0.49      1107
                 offer       0.00      0.00      0.00        38
           aid_related       0.81      0.20      0.32      2686
          medical_help       0.61      0.12      0.20       525
      medical_products       0.66      0.13      0.22       320
     search_and_rescue       0.66      0.21      0.31       184
              security       0.00      0.00      0.00       134
              military       0.00      0.00      0.00       200
                 water       0.59      0.81      0.69       443
                  food       0.76      0.67      0.72       739
               shelter       0.84      0.31      0.45       575
              clothing       0.76      0.28      0.41       100
                 money       0.39      0.08      0.13       157
        missing_people       0.66      

C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
# Pipeline 2: Naive Bayes classifier
pipeline_nbc = Pipeline([
        ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
        ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
        ('tfidf_transformer', TfidfTransformer())]))
        ])),
        ('classifier', MultiOutputClassifier(MultinomialNB()))])

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [28]:
pipeline_nbc_fit = pipeline_nbc.fit(X_train, y_train)

In [29]:
y_pred_train_nbc = pipeline_nbc_fit.predict(X_train)
y_pred_test_nbc  = pipeline_nbc_fit.predict(X_test)

In [30]:
# training data
print('\n',classification_report(y_train.values, y_pred_train_nbc, target_names=y.columns.values))


                         precision    recall  f1-score   support

               related       0.78      1.00      0.88     14975
               request       0.93      0.22      0.35      3389
                 offer       0.00      0.00      0.00        92
           aid_related       0.87      0.70      0.77      8131
          medical_help       0.00      0.00      0.00      1566
      medical_products       0.20      0.00      0.00       992
     search_and_rescue       0.00      0.00      0.00       537
              security       0.00      0.00      0.00       363
              military       0.00      0.00      0.00       670
                 water       0.00      0.00      0.00      1239
                  food       0.76      0.02      0.04      2228
               shelter       0.29      0.00      0.00      1736
              clothing       0.00      0.00      0.00       298
                 money       0.00      0.00      0.00       465
        missing_people       0.00    

C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# test data
print(classification_report(y_test.values, y_pred_test_nbc, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      4931
               request       0.90      0.14      0.25      1085
                 offer       0.00      0.00      0.00        26
           aid_related       0.77      0.58      0.66      2729
          medical_help       0.00      0.00      0.00       518
      medical_products       1.00      0.00      0.01       321
     search_and_rescue       0.00      0.00      0.00       187
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       190
                 water       0.00      0.00      0.00       433
                  food       0.36      0.01      0.01       695
               shelter       0.00      0.00      0.00       578
              clothing       0.00      0.00      0.00       107
                 money       0.00      0.00      0.00       139
        missing_people       0.00      

C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\FELIPE\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [32]:
m = pickle.dumps('classifier.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.